In [ ]:
from IPython.utils import capture
with capture.capture_output() as cap:
  import os, subprocess, sys, fileinput
  import concurrent.futures
  from concurrent.futures import ThreadPoolExecutor, as_completed
  from IPython.display import clear_output
  from google.colab import drive
  if not os.path.exists("/content/ltrain"):
    drive.mount('/content/drive')
    %cd /content/
    clear_output()
    print("установка зависимостей")
    os.makedirs("/content/models", exist_ok=True)
    os.makedirs("/content/drive/MyDrive/LORA", exist_ok=True)
    base_path = '/content/drive/MyDrive/LORA/'
    wget = "wget -nv -t 10 --show-progress --progress=bar -q --content-disposition "
    debs = [
      f"{wget} 'https://civitai.com/api/download/models/127207?type=Model&format=SafeTensor&size=pruned&fp=fp16' -O /content/models/Juggernaut.safetensors",
      f"{wget} 'https://civitai.com/api/download/models/143906?type=Model&format=SafeTensor&size=pruned&fp=fp16' -O /content/models/epiCRealism.safetensors",
      "git clone https://github.com/PR0LAPSE/lora_colab ltrain",
      f"{wget} https://huggingface.co/utnah/deps/resolve/main/db_deps.7z",
    ]
    def colab_downgrade(deb):
      subprocess.getoutput(deb)
    with ThreadPoolExecutor(max_workers=len(debs)) as executor:
      futures = [executor.submit(colab_downgrade, deb) for deb in debs]
      for future in futures:
        future.result()
    !7z -bso0 -bd -mmt4 -slp -y x /content/db_deps.7z -o/usr/local/lib/python3.10/dist-packages > /dev/null 2>&1
    !rm /content/db_deps.7z


    wget = "wget -nv -t 10 --show-progress --progress=bar:force -q --content-disposition"
    def from_repo(filename, path=None):
        if path is not None:
            get_ipython().system(f"{wget} {filename} -P {path}")
        else:
            get_ipython().system(f"{wget} {filename}")

    from_repo("https://huggingface.co/2ch/colab/resolve/main/libtcmalloc_minimal.so.4", "/lib/")
    while (os.path.getsize('/lib/libtcmalloc_minimal.so.4') < 300000) or (not os.path.exists('/lib/libtcmalloc_minimal.so.4')):
        from_repo("https://huggingface.co/2ch/colab/resolve/main/libtcmalloc_minimal.so.4", "/lib/")

    os.environ["LD_PRELOAD"]='/lib/libtcmalloc_minimal.so.4'
    os.environ["PYTORCH_CUDA_ALLOC_CONF"]='garbage_collection_threshold:0.9,max_split_size_mb:512'
    os.environ["SAFETENSORS_FAST_GPU"]='1'
    os.environ["CUDA_MODULE_LOADING"]="LAZY"
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
    os.environ["PYTHONWARNINGS"] = "ignore"
    os.environ.setdefault('ACCELERATE', 'True')
    os.environ.setdefault('ATTN_PRECISION', 'fp16')
    os.environ.setdefault('PYTORCH_CUDA_ALLOC_CONF', 'garbage_collection_threshold:0.8,max_split_size_mb:512')
    os.environ.setdefault('CUDA_LAUNCH_BLOCKING', '0')
    os.environ.setdefault('CUDA_CACHE_DISABLE', '0')
    os.environ.setdefault('CUDA_AUTO_BOOST', '1')
    os.environ.setdefault('CUDA_MODULE_LOADING', 'LAZY')
    os.environ.setdefault('CUDA_DEVICE_DEFAULT_PERSISTING_L2_CACHE_PERCENTAGE_LIMIT', '0')
    os.environ.setdefault('GRADIO_ANALYTICS_ENABLED', 'False')
    os.environ.setdefault('NUMEXPR_MAX_THREADS', '16')
    os.environ.setdefault('PYTHONHTTPSVERIFY', '0')
    os.environ.setdefault('HF_HUB_DISABLE_TELEMETRY', '1')
    os.environ.setdefault('UVICORN_TIMEOUT_KEEP_ALIVE', '60')

    clear_output()
    if загрузить_с_пк:
      from google.colab import files
      import shutil
      folders = ['input', 'output', 'log', 'config']
      for folder in folders:
        os.makedirs(base_path + folder, exist_ok=True)
      input_folder_path = base_path + folders[0]
      print(f"загрузка изображений в {input_folder_path}")
      uploaded = files.upload()
      num_files = len([k for k in uploaded.keys() if '.txt' not in k])
      CONST = 1500
      if int(CONST / num_files) < step_limit:
        num_f = step_limit
      else:
        num_f = int(CONST / num_files)
      num_input = input_folder_path + f'/{num_f}_{trigger_word}'
      os.makedirs(num_input, exist_ok=True)
      for filename in uploaded.keys():
        dst_path = os.path.join(num_input, filename)
        shutil.move(filename, dst_path)
  %cd /content/ltrain
  if Ngrok_token!="":
    from pyngrok import ngrok, conf
    ngrok.kill()
    srv=ngrok.connect(7860, pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token) , bind_tls=True).public_url
    for line in fileinput.input('/content/ltrain/ltrain.py', inplace=True):
      if line.strip().startswith("print('Load CSS...')"):
        line = f"            print('Load CSS...')\n            print('Running on ngrok URL:  {srv}')\n"
      sys.stdout.write(line)
  else:
    for line in fileinput.input('/content/ltrain/ltrain.py', inplace=True):
      if line.strip().startswith("print('Running on ngrok URL:"):
        line = f""
      sys.stdout.write(line)
  clear_output()
  !python /content/ltrain/ltrain.py > /content/out.txt